In [1]:
''' Outdated models '''
# MODEL_NAME='lucyd-edof.pth'
# MODEL_NAME='lucyd-edof-11.pth'

''' New/relevant models '''
# MODEL_NAME='lucyd-small-phantoms.pth' # original model trained on small phantoms
# MODEL_NAME='lucyd-psf-sim.pth' # improved EDOF blur simlation (3d projections + microscopy PSF simulation)
MODEL_NAME='lucyd-psf-sim-2.pth' # improved EDOF blur simlation (slightly less patchy results, trained with finer structures)

In [13]:
from lucyd import LUCYD, device
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import torch
import time 
import os

import cv2 as cv
import numpy as np

Parameters:

In [3]:
RESIZE=True
RESIZE_RESOLUTION=(2560,2560) # will be ignored if RESIZE=False
BG_CORR = True

model = LUCYD(num_res=1).to(device)
model.load_state_dict(torch.load('models/'+MODEL_NAME))
model.eval()
print('number of parameters: {}'.format(sum(p.numel() for p in model.parameters())))

path = '/mnt/m181 PISCO1/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/PNG/'

number of parameters: 8452


Background correction:

In [ ]:
if BG_CORR:
    files = os.listdir(path)
    imgs = [cv.imread(os.path.join(path, file), cv.IMREAD_GRAYSCALE) for file in files]
    bg = np.max(imgs, axis=0)
    corrected = cv.absdiff(imgs[0], bg)
else:
    pass

In [15]:
####Dataset 
import torchvision.transforms as transforms
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
from torch.nn.functional import pad
import os
from PIL import Image
import torch.nn.functional as F

class UnlabelledImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.img_names = sorted(os.listdir(img_dir))
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_path).convert('L')
        if self.transform:
            image = self.transform(image)
        return image, 0  # Return a dummy label

def collate_fn(batch):
    # Find the max height and width in this batch
    max_height = max(img.shape[1] for img, _ in batch)
    max_width = max(img.shape[2] for img, _ in batch)

    # Ensure max_height and max_width are even numbers
    max_height += max_height % 2
    max_width += max_width % 2

    # Add padding to images in this batch and collate them into a single tensor
    batch = [F.pad(img, (0, max_width - img.shape[2], 0, max_height - img.shape[1]), mode='constant', value=0) for img, _ in batch]
    return torch.stack(batch), _ 

transform = transforms.Compose([
        transforms.Resize((2560,2560)),
        #transforms.RandomResizedCrop(224),
        #transforms.RandomHorizontalFlip(),
        #transforms.RandomRotation(degrees=(0, 180)),
        transforms.Grayscale(),
        transforms.ToTensor(),
        #transforms.Normalize([0.5,], [0.5,])
    ])

# Load the unseen dataset
dataset = UnlabelledImageDataset('/home/plankton/Data/M181_test_set', transform=transform)

# Create a data loader
images = DataLoader(dataset, batch_size=1, collate_fn=collate_fn)

In [7]:
print(torch.cuda.memory_allocated())
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated())

209834496
209834496


: 

In [32]:
################
from tqdm import tqdm
# NAME OF THE IMAGE YOU WANT TO DECONVOLVE
#path = '/mnt/m181 PISCO1/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/PNG/'
index = 0
with torch.no_grad():
    for image, _ in images:
        start_time = time.time()

        #filename = os.path.join(path,file)
        #result_fn = os.path.join('/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543',file)
        result_fn = os.path.join('/home/plankton/Results/M181/Deconv/batch_test',str(index)+'.png')
        index += 1 

        ################

            #if RESIZE:
            #    x = np.array(Image.open(filename).convert('L').resize(RESIZE_RESOLUTION))
            #else:
            #    x = np.array(Image.open(filename).convert('L'))
        x = image

            #if x.shape[0] % 2 != 0: x = x[1:]
            #if x.shape[1] % 2 != 0: x = x[:,1:]
            # x = (x - np.min(x))/(np.max(x) - np.min(x))
            #x = x/255

        x_torch = x.to(device)
        #torch.cuda.synchronize()        
            #x_torch = x_torch.unsqueeze(0).unsqueeze(0)
        print('Time to load image: {} seconds'.format(time.time() - start_time))
        start_time = time.time()
        y_hat, y_k, update = model(x_torch.float())
        torch.cuda.synchronize()
        print('Time to deconvolve: {} seconds'.format(time.time() - start_time))
        start_time = time.time()
            # plt.figure(figsize=(20,20))
            # plt.subplot(1,2,1)
        #y = x_torch.detach().cpu().numpy()[0,0]
        #print('Time to detach y image: {} seconds'.format(time.time() - start_time))
            # plt.imshow(y, cmap='gray')
            # plt.title('Input')
            # plt.subplot(1,2,2)
        x = y_hat.detach().cpu().numpy()[0,0]
        print('Time to cpu x image: {} seconds'.format(time.time() - start_time))
        #x = x.numpy()[0,0]
        #print('Time to numpy x image: {} seconds'.format(time.time() - start_time))
        start_time = time.time()

            # plt.imshow(x, cmap='gray', vmin=0)
            # plt.title('Output')
            # plt.show()
        x = x*255
        cv.imwrite(result_fn, x)

        #Image.fromarray(x).convert('L').save(result_fn)
        print('Time to save image: {} seconds'.format(time.time() - start_time))


Time to load image: 0.017263412475585938 seconds
Time to deconvolve: 0.2676856517791748 seconds
Time to cpu x image: 0.015232563018798828 seconds
Time to save image: 0.1028437614440918 seconds
Time to load image: 0.0032896995544433594 seconds
Time to deconvolve: 0.1942272186279297 seconds
Time to cpu x image: 0.003014802932739258 seconds
Time to save image: 0.10143637657165527 seconds
Time to load image: 0.0033359527587890625 seconds
Time to deconvolve: 0.19482207298278809 seconds
Time to cpu x image: 0.015477895736694336 seconds
Time to save image: 0.10285353660583496 seconds
Time to load image: 0.003224611282348633 seconds
Time to deconvolve: 0.19460773468017578 seconds
Time to cpu x image: 0.0029146671295166016 seconds
Time to save image: 0.10470032691955566 seconds
Time to load image: 0.0029320716857910156 seconds
Time to deconvolve: 0.19460582733154297 seconds
Time to cpu x image: 0.014861345291137695 seconds
Time to save image: 0.10429239273071289 seconds
Time to load image: 0.00

KeyboardInterrupt: 

In [ ]:
index = 0
with torch.no_grad():
    for image, _ in images:
        start_time = time.time()

        result_fn = os.path.join('/home/plankton/Results/M181/Deconv/batch_test',str(index)+'.png')
        index += 1 

        x = image

        x_torch = x.to(device)
        print('Time to load image: {} seconds'.format(time.time() - start_time))
        start_time = time.time()
        y_hat, y_k, update = model(x_torch.float())
        print('Time to deconvolve: {} seconds'.format(time.time() - start_time))
        start_time = time.time()
            # plt.figure(figsize=(20,20))
            # plt.subplot(1,2,1)
        #y = x_torch.detach().cpu().numpy()[0,0]
        #print('Time to detach y image: {} seconds'.format(time.time() - start_time))
            # plt.imshow(y, cmap='gray')
            # plt.title('Input')
            # plt.subplot(1,2,2)
        x = y_hat.detach().cpu().numpy()[0,0]
        #print('Time to detach x image: {} seconds'.format(time.time() - start_time))
            # plt.imshow(x, cmap='gray', vmin=0)
            # plt.title('Output')
            # plt.show()
        x = x*255
        cv.imwrite(result_fn, x)

        #Image.fromarray(x).convert('L').save(result_fn)
        print('Time to save image: {} seconds'.format(time.time() - start_time))

In [27]:
#!cd /home/plankton/PISCO_Software/MaxSegmenter
from MaxSegmenterModule import run_segmenter

run_segmenter(
        #source_folder="C:/Users/timka/Documents/Arbeit/PeruTest",
        source_folder="/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/",
        save_crops=False,
        save_marked_imgs=False,
        save_bg_corr_imgs=True,
        min_area_to_segment=1000,
        min_area_to_save=1000,
        save_path="/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/bg_corr/",
        #save_path="C:/Users/timka/Documents/Arbeit/Results",
        equalize_hist=True,
        resize=True,
        clear_save_path=True,
        bg_size=5,
        max_threads=10,
        n_sigma=2, #2 for M181 images, 1.5 for Cusco Peru
        n_cores=8,
        mask_imgs=True,
        mask_radius_offset=200,
    )

unprocessed directory found!
remaining images: 20522
Start segmenting
Mask radius is: 1302
10% done
30% done
40% done
50% done
70% done
80% done
100% done
Total time: 1610.5983849840704, Avg time per img: 0.07848155077400207


In [26]:
%cd /home/plankton/PISCO_Software/MaxSegmenter
%pwd

/home/plankton/PISCO_Software/MaxSegmenter


'/home/plankton/PISCO_Software/MaxSegmenter'

In [28]:
run_segmenter(
        #source_folder="C:/Users/timka/Documents/Arbeit/PeruTest",
        source_folder="/mnt/m181 PISCO1/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/PNG/",
        save_crops=False,
        save_marked_imgs=False,
        save_bg_corr_imgs=True,
        min_area_to_segment=1000,
        min_area_to_save=1000,
        save_path="/home/plankton/Results/M181/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/",
        #save_path="C:/Users/timka/Documents/Arbeit/Results",
        equalize_hist=True,
        resize=True,
        clear_save_path=True,
        bg_size=5,
        max_threads=10,
        n_sigma=2, #2 for M181 images, 1.5 for Cusco Peru
        n_cores=8,
        mask_imgs=True,
        mask_radius_offset=200,
    )

unprocessed directory found!
remaining images: 20521
Start segmenting
Mask radius is: 2820
No progress detected, terminating status update...
Total time: 2011.0037388849305, Avg time per img: 0.09799735582500514


In [30]:
################
# NAME OF THE IMAGE YOU WANT TO DECONVOLVE
path = '/home/plankton/Results/M181/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/Images/'
for file in os.listdir(path):

    filename = os.path.join(path,file)
    result_fn = os.path.join('/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543_bg_corr',file)
    ################

    with torch.no_grad():
        if RESIZE:
            x = np.array(Image.open(filename).convert('L').resize(RESIZE_RESOLUTION))
        else:
            x = np.array(Image.open(filename).convert('L'))

        if x.shape[0] % 2 != 0: x = x[1:]
        if x.shape[1] % 2 != 0: x = x[:,1:]
        # x = (x - np.min(x))/(np.max(x) - np.min(x))
        x = x/255

        x_torch = torch.from_numpy(x).to(device)
        x_torch = x_torch.unsqueeze(0).unsqueeze(0)

        start_time = time.time()
        y_hat, y_k, update = model(x_torch.float())
        print('Time to deconvolve: {} seconds'.format(time.time() - start_time))

        # plt.figure(figsize=(20,20))
        # plt.subplot(1,2,1)
        y = x_torch.detach().cpu().numpy()[0,0]
        # plt.imshow(y, cmap='gray')
        # plt.title('Input')
        # plt.subplot(1,2,2)
        x = y_hat.detach().cpu().numpy()[0,0]
        # plt.imshow(x, cmap='gray', vmin=0)
        # plt.title('Output')
        # plt.show()

        Image.fromarray(x*255.).convert('L').save(result_fn)

Time to deconvolve: 0.11010146141052246 seconds
Time to deconvolve: 0.007553815841674805 seconds
Time to deconvolve: 0.00644230842590332 seconds
Time to deconvolve: 0.007519721984863281 seconds
Time to deconvolve: 0.005121707916259766 seconds
Time to deconvolve: 0.006467103958129883 seconds
Time to deconvolve: 0.007014036178588867 seconds
Time to deconvolve: 0.013376712799072266 seconds
Time to deconvolve: 0.006436824798583984 seconds
Time to deconvolve: 0.0051381587982177734 seconds
Time to deconvolve: 0.005406618118286133 seconds
Time to deconvolve: 0.005468845367431641 seconds
Time to deconvolve: 0.0069620609283447266 seconds
Time to deconvolve: 0.005045890808105469 seconds
Time to deconvolve: 0.00690913200378418 seconds
Time to deconvolve: 0.006078958511352539 seconds
Time to deconvolve: 0.007961511611938477 seconds
Time to deconvolve: 0.005834341049194336 seconds
Time to deconvolve: 0.00554203987121582 seconds
Time to deconvolve: 0.0061244964599609375 seconds
Time to deconvolve: 0